<h1>Notes From PostgreSQL Documentation</h1>

<h3>Installing Dependencies</h3>

<b>Install dependencies and after installment finished restart the notebook.</b>

In [1]:
#!pip install jupyter ipython-sql jupyterlab_sql_editor[all]

<h3>Initializing Notebook and Connections</h3>

<b>This steps will be executed every start/restart of kernel or notebook</b>

In [2]:
%load_ext sql

In [3]:
%config SqlMagic.style = '_DEPRECATED_DEFAULT'

Connecting to DB
Usage: &lt;db_vendor&gt;://[username:password]@&lt;serverip&gt;/&lt;dbname&gt;

In [4]:
%sql postgresql://postgres@localhost/mydb

Testing connection

In [5]:
%sql select 'CONNECTION SUCCESSFULLY INITIALIZED' as result

 * postgresql://postgres@localhost/mydb
1 rows affected.


result
CONNECTION SUCCESSFULLY INITIALIZED


<h2>Chapter 5. Data Definition</h2>

### 5.1. Table Basics

##### a. Creating Table

For details check reference <a href="https://www.postgresql.org/docs/current/sql-createtable.html">https://www.postgresql.org/docs/current/sql-createtable.html</a>

In [6]:
%%sql
DROP TABLE IF EXISTS products;
CREATE TABLE products (
	product_no integer,
	name text,
	price numeric
);

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.DependentObjectsStillExist) HATA:  diğer nesnelerin ona bağlı olması nedeniyle products tablosu kaldırılamıyor
DETAIL:  orders tablosu üzerinde orders_product_no_fkey kısıtlaması, products tablosu nesnesine bağlıdır
HINT:  Bağlı nesneleri de kaldırmak için DROP ... CASCADE kullanın.

[SQL: DROP TABLE IF EXISTS products;]
(Background on this error at: https://sqlalche.me/e/20/2j85)


##### b. Deleting Table

For details check reference <a href="https://www.postgresql.org/docs/current/sql-droptable.html">https://www.postgresql.org/docs/current/sql-droptable.html</a>

- IF EXISTS is optional. This statement prevents throwing error when table does not exist in DB.

In [7]:
%%sql
DROP TABLE IF EXISTS products;

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.DependentObjectsStillExist) HATA:  diğer nesnelerin ona bağlı olması nedeniyle products tablosu kaldırılamıyor
DETAIL:  orders tablosu üzerinde orders_product_no_fkey kısıtlaması, products tablosu nesnesine bağlıdır
HINT:  Bağlı nesneleri de kaldırmak için DROP ... CASCADE kullanın.

[SQL: DROP TABLE IF EXISTS products;]
(Background on this error at: https://sqlalche.me/e/20/2j85)


##### c. Creating Table with DEFAULT value

In [8]:
%%sql
DROP TABLE IF EXISTS products;
CREATE TABLE products (
	product_no integer,
	name text,
	price numeric DEFAULT 9.99,
	date timestamp DEFAULT CURRENT_TIMESTAMP
);

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.DependentObjectsStillExist) HATA:  diğer nesnelerin ona bağlı olması nedeniyle products tablosu kaldırılamıyor
DETAIL:  orders tablosu üzerinde orders_product_no_fkey kısıtlaması, products tablosu nesnesine bağlıdır
HINT:  Bağlı nesneleri de kaldırmak için DROP ... CASCADE kullanın.

[SQL: DROP TABLE IF EXISTS products;]
(Background on this error at: https://sqlalche.me/e/20/2j85)


### 5.3. Identity Columns

##### a. Creating table with IDENTITY column

- When using <i>IDENTITY</i> column with <i>GENERATED</i> it means value of column will be created automatically
- <i>GENERATED ALWAYS</i> means you can not insert manual values when adding/updating for this field.
- <i>GENERATED BY DEFAULT</i> means you can insert manual values when adding/updating for this field
- When using <i>GENERATED BY DEFAULT</i> you must be careful:
    - Generating values will be using a sequential. That means it starts from 1 and goes on.
    - If identity column is described as UNIQUE:
        - You insert a record without inserting a manual value for identity column.
        - After that you give identity column value by hand and that value is 2.
            - It will be successfully inserted.
            - But sequence value is still remains as 2.
        - Then you inserted a record without inserting a manual value for identity column again.
            - Since sequence value is 2 and there is a record also has valued 2, it will throw duplicated value error.
    - If identity column is not described as UNIQUE:
        - You insert a record without inserting a manual value for identity column.
        - After that you give identity column value by hand and that value is 2.
            - It will be successfully inserted.
            - But sequence value is still remains as 2.
        - Then you inserted a record without inserting a manual value for identity column again.
            - Since sequence value is 2 it will insert record with identity column has value as 2.
            - But there are two record which their identity columns has value as 2.
- Even when using <i>GENERATED ALWAYS</i> there are duplicated value or error risks:
    - By using OVERRIDING SYSTEM VALUE, you can enter identity column values manually.
    - Same cases above (GENERATED BY DEFAULT) are also effective for this. 

##### b. Example usage of <i>GENERATED ALWAYS</i>

- Creating table with GENERATED ALWAYS AS IDENTITY column

In [9]:
%%sql
DROP TABLE IF EXISTS people;
CREATE TABLE people (
	id bigint GENERATED ALWAYS AS IDENTITY,
	name varchar(80),
	address varchar(80)
);

 * postgresql://postgres@localhost/mydb
Done.
Done.


[]

- Record will be inserted successfully and id will be automatically assigned as 1

In [10]:
%%sql 
INSERT INTO people (name, address) VALUES ('A', 'foo');
SELECT * FROM people;

 * postgresql://postgres@localhost/mydb
1 rows affected.
1 rows affected.


id,name,address
1,A,foo


- Throws error because of id column is generated always and given an id value

In [11]:
%sql INSERT INTO people (id, name, address) VALUES (2, 'B', 'bar');

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.GeneratedAlways) HATA:  cannot insert a non-DEFAULT value into column "id"
DETAIL:  Column "id" is an identity column defined as GENERATED ALWAYS.
HINT:  Geçersiz kılmak (override) için OVERRIDING SYSTEM VALUE kullanın.

[SQL: INSERT INTO people (id, name, address) VALUES (2, 'B' , 'bar' );]
(Background on this error at: https://sqlalche.me/e/20/f405)


- Will not throw error and insert record successfully

In [12]:
%%sql 
INSERT INTO people (id, name, address) OVERRIDING SYSTEM VALUE VALUES (2, 'B', 'bar');

 * postgresql://postgres@localhost/mydb
1 rows affected.


[]

- Insert will be successful but since sequence for id value is still 2 and there is a record which id is 2, there is two record which has id 2.

In [13]:
%%sql 
INSERT INTO people (name, address) VALUES ('C', 'baz');
SELECT * FROM people;

 * postgresql://postgres@localhost/mydb
1 rows affected.
3 rows affected.


id,name,address
1,A,foo
2,B,bar
2,C,baz


##### c. Example usage of <i>GENERATED BY DEFAULT with UNIQUE descriptor</i>

- Creating table with GENERATED BY DEFAULT AS IDENTITY column

In [14]:
%%sql
DROP TABLE IF EXISTS people;
CREATE TABLE people (
	id bigint UNIQUE GENERATED BY DEFAULT AS IDENTITY,
	name varchar(80),
	address varchar(80)
);

 * postgresql://postgres@localhost/mydb
Done.
Done.


[]

- First record will be inserted successfully and id will be automatically assigned as 1.
- Second record will be inserted successfully.

In [15]:
%%sql 
INSERT INTO people (name, address) VALUES ('A', 'foo');
INSERT INTO people (id, name, address) VALUES (2, 'B', 'bar');
SELECT * FROM people;

 * postgresql://postgres@localhost/mydb
1 rows affected.
1 rows affected.
2 rows affected.


id,name,address
1,A,foo
2,B,bar


- Error will be thrown because sequence for id value is still 2 and there is a record which id is 2

In [16]:
%%sql 
INSERT INTO people (name, address) VALUES ('C', 'baz');

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.UniqueViolation) HATA:  tekrar eden kayıt, "people_id_key" tekil kısıtlamasını ihlal etmektedir
DETAIL:  "(id)=(2)" anahtarı zaten mevcut

[SQL: INSERT INTO people (name, address) VALUES ('C', 'baz');]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


##### d. Example usage of <i>GENERATED BY DEFAULT without UNIQUE descriptor</i>

- Creating table with GENERATED BY DEFAULT AS IDENTITY column

In [17]:
%%sql
DROP TABLE IF EXISTS people;
CREATE TABLE people (
	id bigint GENERATED BY DEFAULT AS IDENTITY,
	name varchar(80),
	address varchar(80)
);

 * postgresql://postgres@localhost/mydb
Done.
Done.


[]

- First record will be inserted successfully and id will be automatically assigned as 1.
- Second record will be inserted successfully.

In [18]:
%%sql 
INSERT INTO people (name, address) VALUES ('A', 'foo');
INSERT INTO people (id, name, address) VALUES (2, 'B', 'bar');
SELECT * FROM people;

 * postgresql://postgres@localhost/mydb
1 rows affected.
1 rows affected.
2 rows affected.


id,name,address
1,A,foo
2,B,bar


- Insert will be successful but since sequence for id value is still 2 and there is a record which id is 2, there is two record which has id 2.

In [19]:
%%sql 
INSERT INTO people (name, address) VALUES ('C', 'baz');
SELECT * FROM people;

 * postgresql://postgres@localhost/mydb
1 rows affected.
3 rows affected.


id,name,address
1,A,foo
2,B,bar
2,C,baz


##### e. Usage of DEFAULT value

- DEFAULT also can be used to specify the sequence explicitly.
- Especially in UPDATE command when update generated column value and get it from sequence, you can use DEFAULT.

- Creating table has generated column

In [20]:
%%sql
DROP TABLE IF EXISTS people;
CREATE TABLE people (
	id bigint GENERATED ALWAYS AS IDENTITY,
	name varchar(80),
	address varchar(80)
);

 * postgresql://postgres@localhost/mydb
Done.
Done.


[]

- Inserting values.
- When inserting values id will be fetched from a sequence.
- First record has id 1, and second has id 2.
- After insertions completed, current value of sequence will be 3.

In [21]:
%%sql
INSERT INTO people (name, address) VALUES ('A', 'foo');
INSERT INTO people (name, address) VALUES ('B', 'bar');
SELECT * FROM people;

 * postgresql://postgres@localhost/mydb
1 rows affected.
1 rows affected.
2 rows affected.


id,name,address
1,A,foo
2,B,bar


- Inserting value using DEFAULT.
- Using DEFAULT means use sequence to insert value.
- So the value will be current value of sequence (3) and current value of sequence increases.

In [22]:
%%sql 
INSERT INTO people (id, name, address) VALUES (DEFAULT, 'C', 'baz');
SELECT * FROM people;

 * postgresql://postgres@localhost/mydb
1 rows affected.
3 rows affected.


id,name,address
1,A,foo
2,B,bar
3,C,baz


- DEFAULT value also can be used in UPDATE statement.
- When using update statement, id value will be fetched from sequence still.
- So new id of third record will be 4.
- The other use of DEFAULT is in name column. Default value of name is NULL because it is not specified a default value.

In [23]:
%%sql
UPDATE people SET id=DEFAULT, name=DEFAULT, address='qwe' where id = 3;
SELECT * FROM people; 

 * postgresql://postgres@localhost/mydb
1 rows affected.
3 rows affected.


id,name,address
1,A,foo
2,B,bar
4,None,qwe


### 5.4. Generated Columns

- Syntax is GENERATED [ALWAYS|BY DEFAULT] AS ... STORED
- Creates a calculated column by using a column as base column.
- Example usages:
    - When you have to calculate a value using a column frequently, then you can store the calculated value in another column.
        - By that way you can also index that column and make optimizations.
    - When calculating, if base column is NULL then generated column will be also NULL.
    - (From documentation) Generated columns maintain access privileges separately from their underlying base columns.
        - So, it is possible to arrange it so that a particular role can read from a generated column but not from the underlying base columns.
        - For example you can hide email value from users except current user is admin (for simplicity current_user used instead of role based query).
            - To achive this there are 2 columns for email: email and visible_email
            - visible_email will be generated using email.
            - When getting visible_email if current user is admin email will be seen, but is not admin ***** will be seen in results.
            - To prevent other users get email by using email column, user rights will be revoked.
            - ```sql
                CREATE TABLE users (
                    id SERIAL PRIMARY KEY,
                    email VARCHAR(200) NOT NULL,
                    visible_email VARCHAR(200) GENERATED ALWAYS AS (
                        CASE 
                            WHEN current_user = 'admin' THEN email
                            ELSE '*****'
                        END
                    ) STORED
                );
                REVOKE SELECT ON users FROM PUBLIC;
                GRANT SELECT (id, visible_email) ON users TO PUBLIC;
                GRANT SELECT (id, email, visible_email) ON users TO admin;
              ```
     

##### a. Creating table with GENERATED column

In [24]:
%%sql
DROP TABLE IF EXISTS people;
CREATE TABLE people (
	id bigint PRIMARY KEY GENERATED ALWAYS AS IDENTITY,
	name varchar(80),
	address varchar(80),
	height_cm numeric,
	height_in numeric GENERATED ALWAYS AS (height_cm / 2.54) STORED
);

 * postgresql://postgres@localhost/mydb
Done.
Done.


[]

- Inserting values to table with GENERATED column.
- See that when height_cm is null, generated column is also null.

In [25]:
%%sql
INSERT INTO people (name, address, height_cm) VALUES ('A', 'foo', 170); 
INSERT INTO people (name, address) VALUES ('B', 'bar');
SELECT * FROM people;

 * postgresql://postgres@localhost/mydb
1 rows affected.
1 rows affected.
2 rows affected.


id,name,address,height_cm,height_in
1,A,foo,170,66.9291338582677165
2,B,bar,None,None


### 5.5. Constraints

#### 5.5.1. Check Constraints

##### a. Creating table with check constraint

In [26]:
%%sql
DROP TABLE IF EXISTS products;
CREATE TABLE products (
	product_no integer,
	name text,
	price numeric CHECK (price > 0)
);

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.DependentObjectsStillExist) HATA:  diğer nesnelerin ona bağlı olması nedeniyle products tablosu kaldırılamıyor
DETAIL:  orders tablosu üzerinde orders_product_no_fkey kısıtlaması, products tablosu nesnesine bağlıdır
HINT:  Bağlı nesneleri de kaldırmak için DROP ... CASCADE kullanın.

[SQL: DROP TABLE IF EXISTS products;]
(Background on this error at: https://sqlalche.me/e/20/2j85)


- Inserting record to table has check constraint
- Operation will throw error because of price should be greater than zero.

In [27]:
%sql INSERT INTO products VALUES (1, 'product-1', 0);

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.UniqueViolation) HATA:  tekrar eden kayıt, "products_pkey" tekil kısıtlamasını ihlal etmektedir
DETAIL:  "(product_no)=(1)" anahtarı zaten mevcut

[SQL: INSERT INTO products VALUES (1, 'product-1' , 0);]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


##### b. Creating table with check constraint which has name

- Giving a constraint name clarifies error messages and allows you to refer it when constraint needed to change

In [28]:
%%sql
DROP TABLE IF EXISTS products;
CREATE TABLE products (
	product_no integer,
	name text,
	price numeric CONSTRAINT positive_price CHECK (price > 0)
);

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.DependentObjectsStillExist) HATA:  diğer nesnelerin ona bağlı olması nedeniyle products tablosu kaldırılamıyor
DETAIL:  orders tablosu üzerinde orders_product_no_fkey kısıtlaması, products tablosu nesnesine bağlıdır
HINT:  Bağlı nesneleri de kaldırmak için DROP ... CASCADE kullanın.

[SQL: DROP TABLE IF EXISTS products;]
(Background on this error at: https://sqlalche.me/e/20/2j85)


- Inserting record to table has named check constraint
- Operation will throw error because of price should be greater than zero.
- See that error message includes constraint name (positive_price).

In [29]:
%sql INSERT INTO products VALUES (1, 'product-1', 0);

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.UniqueViolation) HATA:  tekrar eden kayıt, "products_pkey" tekil kısıtlamasını ihlal etmektedir
DETAIL:  "(product_no)=(1)" anahtarı zaten mevcut

[SQL: INSERT INTO products VALUES (1, 'product-1' , 0);]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


##### c. Removing constraint from table and adding new constraint

- First creating table with constraint and give a name to constraint.

In [30]:
%%sql
DROP TABLE IF EXISTS products;
CREATE TABLE products (
	product_no integer,
	name text,
	price numeric CONSTRAINT positive_price CHECK (price > 0)
);

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.DependentObjectsStillExist) HATA:  diğer nesnelerin ona bağlı olması nedeniyle products tablosu kaldırılamıyor
DETAIL:  orders tablosu üzerinde orders_product_no_fkey kısıtlaması, products tablosu nesnesine bağlıdır
HINT:  Bağlı nesneleri de kaldırmak için DROP ... CASCADE kullanın.

[SQL: DROP TABLE IF EXISTS products;]
(Background on this error at: https://sqlalche.me/e/20/2j85)


- Removing a constraint and adding new constraint.
- Since constraint has a name it easy to remove using name.
- When not given a name you should find constraint int pg tables. (For more: Google -> PostgreSQL drop constraint with unknown name)

In [31]:
%%sql
ALTER TABLE products DROP CONSTRAINT positive_price;
ALTER TABLE products ADD CONSTRAINT minimum_price CHECK (price >= 10.0);

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.UndefinedObject) HATA:  "positive_price" kısıtlaması "products" nesnesinde mevcut değil

[SQL: ALTER TABLE products DROP CONSTRAINT positive_price;]
(Background on this error at: https://sqlalche.me/e/20/f405)


##### d. Other constraint informations

- Check constraint can be added for multiple columns or full table.
- When adding new constraint to non-empty table, all records should complain new constraint. Otherwise constraint is violated by some row error will be thrown. (This is not always the case; details will be provided later.)

- First recreating table for cleaning constraints.

In [32]:
%%sql
DROP TABLE IF EXISTS products;
CREATE TABLE products (
	product_no integer,
	name text,
	price numeric CONSTRAINT positive_price CHECK (price > 0)
);

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.DependentObjectsStillExist) HATA:  diğer nesnelerin ona bağlı olması nedeniyle products tablosu kaldırılamıyor
DETAIL:  orders tablosu üzerinde orders_product_no_fkey kısıtlaması, products tablosu nesnesine bağlıdır
HINT:  Bağlı nesneleri de kaldırmak için DROP ... CASCADE kullanın.

[SQL: DROP TABLE IF EXISTS products;]
(Background on this error at: https://sqlalche.me/e/20/2j85)


- Adding constraint using multiple columns.

In [33]:
%sql ALTER TABLE products ADD CONSTRAINT product_no_and_price CHECK (product_no > 0 and price > 0);

 * postgresql://postgres@localhost/mydb
Done.


[]

- Adding constraint for all columns.

In [34]:
%sql ALTER TABLE products ADD CONSTRAINT lock_table CHECK (false);

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.CheckViolation) HATA:  check constraint "lock_table" of relation "products" is violated by some row

[SQL: ALTER TABLE products ADD CONSTRAINT lock_table CHECK (false);]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


- Check constraing can be applied only new added or modified records.
- To achieve this NOT VALID should be used.

- First cleaning constraints.

In [35]:
%%sql 
ALTER TABLE products DROP CONSTRAINT IF EXISTS product_no_and_price;
ALTER TABLE products DROP CONSTRAINT IF EXISTS lock_table;

 * postgresql://postgres@localhost/mydb
Done.
Done.


[]

- Inserting values to product table to see effect of new constraints on old values.

In [36]:
%%sql
INSERT INTO products VALUES (1, 'product-1', 10);
INSERT INTO products VALUES (2, 'product-2', 20);
SELECT * FROM products;

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.UniqueViolation) HATA:  tekrar eden kayıt, "products_pkey" tekil kısıtlamasını ihlal etmektedir
DETAIL:  "(product_no)=(1)" anahtarı zaten mevcut

[SQL: INSERT INTO products VALUES (1, 'product-1', 10);]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


- First see when adding new constraing without NOT VALID property.
- It throws error because there are rows which not comply with new constraint.

In [37]:
%sql ALTER TABLE products ADD CONSTRAINT lock_table CHECK (false);

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.CheckViolation) HATA:  check constraint "lock_table" of relation "products" is violated by some row

[SQL: ALTER TABLE products ADD CONSTRAINT lock_table CHECK (false);]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


- Now when using NOT VALID, constraint does not check old values
- So there will be no error thrown.

In [38]:
%sql ALTER TABLE products ADD CONSTRAINT lock_table CHECK (false) NOT VALID;

 * postgresql://postgres@localhost/mydb
Done.


[]

- You can check the constraint for old values manually.
- To do that you can use VALIDATE command.
- This statement throws error if there is any record that violates new constraint.

In [39]:
%sql ALTER TABLE products VALIDATE CONSTRAINT lock_table;

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.CheckViolation) HATA:  check constraint "lock_table" of relation "products" is violated by some row

[SQL: ALTER TABLE products VALIDATE CONSTRAINT lock_table;]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


- Also when inserting new value to table it will throw error.
- Because constraint does not allow new records.

In [40]:
%sql INSERT INTO products VALUES (3, 'product-3', 30);

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.CheckViolation) HATA:  "products" tablosuna girilen yeni satır "lock_table" check kısıtlamasını ihlal ediyor
DETAIL:  Hata veren satır (3, product-3, 30) içeriyor.

[SQL: INSERT INTO products VALUES (3, 'product-3' , 30);]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


#### Constraint property DEFERRABLE (Independent from book)

- For constraints there is also DEFERRABLE property is available.
- DEFERRABLE specifies action timing for constraints.
- It specifies will constraint applied immediately or applied end of transaction.
- Usage is: CONSTRAINT ... [ DEFERRABLE | NOT DEFERRABLE ] [ INITIALLY DEFERRED | INITIALLY IMMEDIATE ]
    - [ DEFERRABLE | NOT DEFERRABLE ]: default value is NOT DEFERRABLE.
    - [ INITIALLY DEFERRED | INITIALLY IMMEDIATE ]: default value is INITIALLY IMMEDIATE.
- INITIALLY DEFERRED means in all operations constraint will be checked immediately before insert/update.
- INITIALLY DEFERRED means checking constraint wil be deferred until end of transaction (commit, rollback or error states).
- Actually this cases will be initial case for DEFERRABLE. That means you can change DEFERRED or IMMEDIATE status in the transaction. But this change is only valid in transaction. Does not affect other transactions.
    - For example when defining foreign key constraint with DEFERRABLE without specifying immediate or deferred, its default value is INITIALLY IMMEDIATE.
        - SET CONSTRAINT ... DEFERRED
    - So when in transaction if foreign key will be given later in the transaction, you can set status to DEFERRED and foreign key constraint will be checked end of transaction.
        - SET CONSTRAINT ... IMMEDIATE
    - The reverse could have also been done. A constraint can be defined INITIALLY DEFERRED and you may want to see result of an operation without waiting end of transaction. So you can change it to IMMEDIATE in the transaction. Then constraint will be checked immediately.
- If NOT DEFERRABLE used, you can not change the defer status in the transaction temporarily.
- Yes it can be changed by using ALTER TABLE statement but if affects all latter transactions. Since this action involves modifying the database schema, it is not recommended to use it indiscriminately.
    - ALTER TABLE ... DROP CONSTRAINT ...;
    - ALTER TABLE ... ADD CONSTRAINT ... FOREIGN KEY (...) REFERENCES ... DEFERRABLE INITIALLY DEFERRED;

- To see the effects in this section auto-commit will be disabled.
- Do not forget to re-open for other sections.

In [41]:
%config SqlMagic.autocommit=False

###### e.1 DEFERRABLE NOT DEFERRABLE usage

- Since default value for deferration is NOT DEFERRABLE, unique_product_no constraint will be NOT DEFERRABLE 

In [42]:
%%sql
DROP TABLE IF EXISTS products;
CREATE TABLE products (
	product_no integer CONSTRAINT unique_product_no UNIQUE,
	name text,
	price numeric
);
COMMIT;

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.DependentObjectsStillExist) HATA:  diğer nesnelerin ona bağlı olması nedeniyle products tablosu kaldırılamıyor
DETAIL:  orders tablosu üzerinde orders_product_no_fkey kısıtlaması, products tablosu nesnesine bağlıdır
HINT:  Bağlı nesneleri de kaldırmak için DROP ... CASCADE kullanın.

[SQL: DROP TABLE IF EXISTS products;]
(Background on this error at: https://sqlalche.me/e/20/2j85)


- Second insertion will throw error immediately without waiting the transaction ends. 
- Since error thrown in transaction, it rolls back and none of the insertions will be reflected in the table.

In [43]:
%%sql
BEGIN;
INSERT INTO products VALUES (1, 'product-1', 10);
INSERT INTO products VALUES (1, 'product-1', 10);
INSERT INTO products VALUES (2, 'product-2', 20);
COMMIT;

 * postgresql://postgres@localhost/mydb
Done.
(psycopg2.errors.CheckViolation) HATA:  "products" tablosuna girilen yeni satır "lock_table" check kısıtlamasını ihlal ediyor
DETAIL:  Hata veren satır (1, product-1, 10) içeriyor.

[SQL: INSERT INTO products VALUES (1, 'product-1', 10);]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


In [44]:
%sql SELECT * FROM products;

 * postgresql://postgres@localhost/mydb
1 rows affected.


product_no,name,price
1,product-1,10


###### e.2 DEFERRABLE INITIALLY DEFERRED usage

- Creating table with INITIALLY DEFERRED statement.

In [45]:
%%sql
DROP TABLE IF EXISTS products;
CREATE TABLE products (
	product_no integer CONSTRAINT unique_product_no UNIQUE DEFERRABLE INITIALLY DEFERRED,
	name text,
	price numeric
);
COMMIT;

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.DependentObjectsStillExist) HATA:  diğer nesnelerin ona bağlı olması nedeniyle products tablosu kaldırılamıyor
DETAIL:  orders tablosu üzerinde orders_product_no_fkey kısıtlaması, products tablosu nesnesine bağlıdır
HINT:  Bağlı nesneleri de kaldırmak için DROP ... CASCADE kullanın.

[SQL: DROP TABLE IF EXISTS products;]
(Background on this error at: https://sqlalche.me/e/20/2j85)


- In this transaction second insertion will not throw error. Because unique constraint check deferred until transaction ends.
- In third statement products with id 1 are deleted.
- And in fourth statement a product with id 1 is inserted to empty table.
- At the end of transaction there is only 1 record which has id 1.
- So transaction commit successfully because all records comply with unique constraint.

In [46]:
%%sql
BEGIN;
INSERT INTO products VALUES (1, 'product-1', 10);
INSERT INTO products VALUES (1, 'product-1', 10);
DELETE FROM products where product_no=1;
INSERT INTO products VALUES (1, 'product-1', 10);
COMMIT;

 * postgresql://postgres@localhost/mydb
Done.
(psycopg2.errors.CheckViolation) HATA:  "products" tablosuna girilen yeni satır "lock_table" check kısıtlamasını ihlal ediyor
DETAIL:  Hata veren satır (1, product-1, 10) içeriyor.

[SQL: INSERT INTO products VALUES (1, 'product-1', 10);]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


- In the end there is only 1 record in table.

In [47]:
%sql SELECT * FROM products;

 * postgresql://postgres@localhost/mydb
1 rows affected.


product_no,name,price
1,product-1,10


- An example that gives an error.
- But the error will be thrown at the end of transaction.

In [48]:
%%sql
BEGIN;
INSERT INTO products VALUES (1, 'product-1', 10);
INSERT INTO products VALUES (1, 'product-1', 10);
COMMIT;

 * postgresql://postgres@localhost/mydb
Done.
(psycopg2.errors.CheckViolation) HATA:  "products" tablosuna girilen yeni satır "lock_table" check kısıtlamasını ihlal ediyor
DETAIL:  Hata veren satır (1, product-1, 10) içeriyor.

[SQL: INSERT INTO products VALUES (1, 'product-1', 10);]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


###### e.3 Changing DEFERRABLE status in transaction

- First creating a table with DEFERRABLE INITIALLY IMMEDIATE

In [49]:
%%sql
DROP TABLE IF EXISTS products;
CREATE TABLE products (
	product_no integer CONSTRAINT unique_product_no UNIQUE DEFERRABLE INITIALLY IMMEDIATE,
	name text,
	price numeric
);
COMMIT;

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.DependentObjectsStillExist) HATA:  diğer nesnelerin ona bağlı olması nedeniyle products tablosu kaldırılamıyor
DETAIL:  orders tablosu üzerinde orders_product_no_fkey kısıtlaması, products tablosu nesnesine bağlıdır
HINT:  Bağlı nesneleri de kaldırmak için DROP ... CASCADE kullanın.

[SQL: DROP TABLE IF EXISTS products;]
(Background on this error at: https://sqlalche.me/e/20/2j85)


- In this transaction after adding a record to table changed CONSTRAINT status to DEFERRED.
- When trying to add new record with same id do not throw error, because constraint deferred until transaction ends.
- Since no error thrown, deleting records with id 1, adding new records with id 2 and 1 execute successfully.
- When transaction ends unique constraint check validates that all records.
- Since all records comply with constraint there is 2 record in the table in the end.

In [50]:
%%sql
BEGIN;
INSERT INTO products VALUES (1, 'product-1', 10);
SET CONSTRAINTS unique_product_no DEFERRED;
INSERT INTO products VALUES (1, 'product-1', 10);
DELETE FROM products WHERE product_no = 1;
INSERT INTO products VALUES (2, 'product-2', 20);
INSERT INTO products VALUES (1, 'product-1', 10);
COMMIT;

 * postgresql://postgres@localhost/mydb
Done.
(psycopg2.errors.CheckViolation) HATA:  "products" tablosuna girilen yeni satır "lock_table" check kısıtlamasını ihlal ediyor
DETAIL:  Hata veren satır (1, product-1, 10) içeriyor.

[SQL: INSERT INTO products VALUES (1, 'product-1', 10);]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


In [51]:
%sql SELECT * FROM products;

 * postgresql://postgres@localhost/mydb
1 rows affected.


product_no,name,price
1,product-1,10


###### e.4 Changing DEFERRABILITY

- Not specified deferrability so it is NOT DEFERRABLE

In [52]:
%%sql
DROP TABLE IF EXISTS products;
CREATE TABLE products (
	product_no integer CONSTRAINT unique_product_no UNIQUE,
	name text,
	price numeric
);
COMMIT;

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.DependentObjectsStillExist) HATA:  diğer nesnelerin ona bağlı olması nedeniyle products tablosu kaldırılamıyor
DETAIL:  orders tablosu üzerinde orders_product_no_fkey kısıtlaması, products tablosu nesnesine bağlıdır
HINT:  Bağlı nesneleri de kaldırmak için DROP ... CASCADE kullanın.

[SQL: DROP TABLE IF EXISTS products;]
(Background on this error at: https://sqlalche.me/e/20/2j85)


- Since unique_product_no is not deferrable, when change constraint to DEFERRED will cause error.

In [53]:
%%sql
BEGIN;
INSERT INTO products VALUES (1, 'product-1', 10);
SET CONSTRAINTS unique_product_no DEFERRED;
COMMIT;

 * postgresql://postgres@localhost/mydb
Done.
(psycopg2.errors.CheckViolation) HATA:  "products" tablosuna girilen yeni satır "lock_table" check kısıtlamasını ihlal ediyor
DETAIL:  Hata veren satır (1, product-1, 10) içeriyor.

[SQL: INSERT INTO products VALUES (1, 'product-1', 10);]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


- To make NOT DEFERRABLE constraint to DEFERRED, ALTER TABLE used.
- This is not temporarily in transaction and affects all latter transactions.

In [54]:
%%sql
BEGIN;
INSERT INTO products VALUES (1, 'product-1', 10);
ALTER TABLE products DROP CONSTRAINT unique_product_no;
ALTER TABLE products ADD CONSTRAINT unique_product_no UNIQUE(product_no) DEFERRABLE INITIALLY DEFERRED;
INSERT INTO products VALUES (1, 'product-1', 10);
DELETE FROM products WHERE product_no = 1;
INSERT INTO products VALUES (2, 'product-2', 20);
INSERT INTO products VALUES (1, 'product-1', 10);
COMMIT;

 * postgresql://postgres@localhost/mydb
Done.
(psycopg2.errors.CheckViolation) HATA:  "products" tablosuna girilen yeni satır "lock_table" check kısıtlamasını ihlal ediyor
DETAIL:  Hata veren satır (1, product-1, 10) içeriyor.

[SQL: INSERT INTO products VALUES (1, 'product-1', 10);]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


In [55]:
%sql SELECT * FROM products;

 * postgresql://postgres@localhost/mydb
1 rows affected.


product_no,name,price
1,product-1,10


In [56]:
%config SqlMagic.autocommit=True

#### 5.5.2. Not-Null Constraints

- <i>NOT NULL</i> constraint is a column constraint. You can not write it as table constraint.
    - <i>product_no integer NOT NULL</i> is correct usage.
    - But there is no usage like this: <del><i>CONSTRAINT not_null NOT NULL (product_no)</i></del>.
- <i>product_no integer NOT NULL</i> and <i>CONSTRAINT product_no_not_null CHECK (product_no IS NOT NULL)</i> is functionally equivalent. But NOT NULL constraint is more optimized.
    - With using CONSTRAINT it became table constraing and while defining explicit name it can be removed bey using name.
        - <i>ALTER TABLE ... DROP CONSTRAINT ...;</i>
- Since <i>NOT NULL</i> is column constraint and it can not be explicitly named, you can remove it with using <i>ALTER TABLE ... ALTER COLUMN ... DROP NUT NULL;</i>
- In column there can be multiple constraints.
    - Example: <i>price numeric NOT NULL CHECK (price > 0)</i>

- NOT NULL example

In [57]:
%%sql
DROP TABLE IF EXISTS products;
CREATE TABLE products (
    product_no integer NOT NULL,
    name text NOT NULL,
    price numeric
);

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.DependentObjectsStillExist) HATA:  diğer nesnelerin ona bağlı olması nedeniyle products tablosu kaldırılamıyor
DETAIL:  orders tablosu üzerinde orders_product_no_fkey kısıtlaması, products tablosu nesnesine bağlıdır
HINT:  Bağlı nesneleri de kaldırmak için DROP ... CASCADE kullanın.

[SQL: DROP TABLE IF EXISTS products;]
(Background on this error at: https://sqlalche.me/e/20/2j85)


- Violating NOT NULL constraint

In [58]:
%sql INSERT INTO products(price) VALUES (10);

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.NotNullViolation) HATA:  null value in column "product_no" of relation "products" violates not-null constraint
DETAIL:  Hata veren satır (null, null, 10) içeriyor.

[SQL: INSERT INTO products(price) VALUES (10);]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


- Remove not null constraint example

In [59]:
%%sql
ALTER TABLE PRODUCTS 
    ALTER COLUMN product_no DROP NOT NULL,
    ALTER COLUMN name DROP NOT NULL;

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.InvalidTableDefinition) HATA:  "product_no" sütunu bir birincil anahtardır

[SQL: ALTER TABLE PRODUCTS 
    ALTER COLUMN product_no DROP NOT NULL,
    ALTER COLUMN name DROP NOT NULL;]
(Background on this error at: https://sqlalche.me/e/20/f405)


- Not violating after dropping NOT NULL constraint

In [60]:
%sql INSERT INTO products(price) VALUES (10);

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.NotNullViolation) HATA:  null value in column "product_no" of relation "products" violates not-null constraint
DETAIL:  Hata veren satır (null, null, 10) içeriyor.

[SQL: INSERT INTO products(price) VALUES (10);]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


- Multiple constraints in column example

In [61]:
%%sql
DROP TABLE IF EXISTS products;
CREATE TABLE products (
    product_no integer NOT NULL,
    name text NOT NULL,
    price numeric NOT NULL CHECK (price > 0)
);

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.DependentObjectsStillExist) HATA:  diğer nesnelerin ona bağlı olması nedeniyle products tablosu kaldırılamıyor
DETAIL:  orders tablosu üzerinde orders_product_no_fkey kısıtlaması, products tablosu nesnesine bağlıdır
HINT:  Bağlı nesneleri de kaldırmak için DROP ... CASCADE kullanın.

[SQL: DROP TABLE IF EXISTS products;]
(Background on this error at: https://sqlalche.me/e/20/2j85)


#### 5.5.3. Unique Constraints

- <i>UNIQUE</i> constraint can be written either as a column constraint or a table constraint.
    - Table constraint: <i>product_no integer CONSTRAINT unique_product_no UNIQUE</i>
        - Table constraint can be removed by using <i>ALTER TABLE ... DROP CONSTRAINT ...;</i>
    - Column constraint: <i>product_no integer CONSTRAINT unique_product_no UNIQUE</i>
        - Column constraint can not be removed by using <del><i>ALTER TABLE ... ALTER COLUMN DROP UNIQUE;</i></del>
        - To remove constraint you first find the constraint name and remove with <i>ALTER TABLE ... DROP CONSTRAINT ...;</i>
- <i>UNIQUE</i> constraint can be written for one column or multiple column group.
    - For one column: <i>UNIQUE (product_no)</i>. This ensures product_no is unique.
    - For multiple column: <i>UNIQUE (a,c)</i>. This ensures a and c pair is unique.
        - For example when (a,c) = (1,1) (a,c) = (1,2) (a,c) = (2,1) can be inserted but (a,c) = (1,1) can not be inserted again.
- When using UNIQUE constraint, a UNIQUE B-tree index will be added to table.
- If you want to add UNIQUE constraint for only filtered rows you hould use partial index:
  - <i>CREATE UNIQUE INDEX unique_active_product_no ON products (product_no) WHERE active = true;</i>
- <i>UNIQUE</i> constraint on NULL values:
    - NULL values will be seen as different in UNIQUE columns. So you can insert multiple NULL values for a UNIQUE column and you get no violation of constraint error.
    - To prevent this situation you can use <i>NULLS NOT DISTINCT</i> statement: "<i>product_no integer UNIQUE NULLS NOT DISTINCT</i>".

##### Creating UNIQUE constraint for single column

In [62]:
%%sql
DROP TABLE IF EXISTS products;
CREATE TABLE products (
    product_no integer UNIQUE,
    name text,
    price numeric
);

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.DependentObjectsStillExist) HATA:  diğer nesnelerin ona bağlı olması nedeniyle products tablosu kaldırılamıyor
DETAIL:  orders tablosu üzerinde orders_product_no_fkey kısıtlaması, products tablosu nesnesine bağlıdır
HINT:  Bağlı nesneleri de kaldırmak için DROP ... CASCADE kullanın.

[SQL: DROP TABLE IF EXISTS products;]
(Background on this error at: https://sqlalche.me/e/20/2j85)


- Violating unique constraint

In [63]:
%%sql
INSERT INTO products VALUES (1, 'name-1', 10);
INSERT INTO products VALUES (1, 'name-1', 10);

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.CheckViolation) HATA:  "products" tablosuna girilen yeni satır "lock_table" check kısıtlamasını ihlal ediyor
DETAIL:  Hata veren satır (1, name-1, 10) içeriyor.

[SQL: INSERT INTO products VALUES (1, 'name-1', 10);]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


##### Creating UNIQUE constraint for column pair

In [64]:
%%sql
DROP TABLE IF EXISTS example;
CREATE TABLE example (
    a integer,
    b integer,
    c integer,
    UNIQUE (a, c)
);

 * postgresql://postgres@localhost/mydb
Done.
Done.


[]

- Inserting values

In [65]:
%%sql
INSERT INTO example (a,c) VALUES ('1', '1');
INSERT INTO example (a,c) VALUES ('1', '2');
INSERT INTO example (a,c) VALUES ('2', '1');

 * postgresql://postgres@localhost/mydb
1 rows affected.
1 rows affected.
1 rows affected.


[]

- Violating constraint

In [66]:
%sql INSERT INTO example (a,c) VALUES ('1', '1');

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.UniqueViolation) HATA:  tekrar eden kayıt, "example_a_c_key" tekil kısıtlamasını ihlal etmektedir
DETAIL:  "(a, c)=(1, 1)" anahtarı zaten mevcut

[SQL: INSERT INTO example (a,c) VALUES ('1', '1' );]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


##### Removing UNIQUE constraint

In [67]:
%%sql
DROP TABLE IF EXISTS products;
CREATE TABLE products (
    product_no integer,
    name text,
    price numeric,
    CONSTRAINT unique_product_no UNIQUE (product_no)
);

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.DependentObjectsStillExist) HATA:  diğer nesnelerin ona bağlı olması nedeniyle products tablosu kaldırılamıyor
DETAIL:  orders tablosu üzerinde orders_product_no_fkey kısıtlaması, products tablosu nesnesine bağlıdır
HINT:  Bağlı nesneleri de kaldırmak için DROP ... CASCADE kullanın.

[SQL: DROP TABLE IF EXISTS products;]
(Background on this error at: https://sqlalche.me/e/20/2j85)


In [68]:
%sql ALTER TABLE products DROP CONSTRAINT unique_product_no;

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.UndefinedObject) HATA:  "unique_product_no" kısıtlaması "products" nesnesinde mevcut değil

[SQL: ALTER TABLE products DROP CONSTRAINT unique_product_no;]
(Background on this error at: https://sqlalche.me/e/20/f405)


##### Creating UNIQUE constraint for filtered records using partial INDEX

In [69]:
%%sql
DROP TABLE IF EXISTS products;
CREATE TABLE products (
    product_no integer,
    name text,
    price numeric,
    active boolean
);

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.DependentObjectsStillExist) HATA:  diğer nesnelerin ona bağlı olması nedeniyle products tablosu kaldırılamıyor
DETAIL:  orders tablosu üzerinde orders_product_no_fkey kısıtlaması, products tablosu nesnesine bağlıdır
HINT:  Bağlı nesneleri de kaldırmak için DROP ... CASCADE kullanın.

[SQL: DROP TABLE IF EXISTS products;]
(Background on this error at: https://sqlalche.me/e/20/2j85)


In [70]:
%sql CREATE UNIQUE INDEX unique_index_products_product_no_active ON products (product_no) WHERE active = true;

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.UndefinedColumn) HATA:  "active" sütunu mevcut değil
LINE 1: ..._product_no_active ON products (product_no) WHERE active = t...
                                                             ^

[SQL: CREATE UNIQUE INDEX unique_index_products_product_no_active ON products (product_no) WHERE active = true;]
(Background on this error at: https://sqlalche.me/e/20/f405)


- Violating constraint

In [71]:
%%sql
INSERT INTO products (product_no, active) VALUES (1, true);
INSERT INTO products (product_no, active) VALUES (2, false);

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.UndefinedColumn) HATA:  "active" kolonu "products" tablosunda mevcut değil
LINE 1: INSERT INTO products (product_no, active) VALUES (1, true);
                                          ^

[SQL: INSERT INTO products (product_no, active) VALUES (1, true);]
(Background on this error at: https://sqlalche.me/e/20/f405)


- This throws error because there is active product 1 

In [72]:
%sql INSERT INTO products (product_no, active) VALUES (1, true);

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.UndefinedColumn) HATA:  "active" kolonu "products" tablosunda mevcut değil
LINE 1: INSERT INTO products (product_no, active) VALUES (1, true);
                                          ^

[SQL: INSERT INTO products (product_no, active) VALUES (1, true);]
(Background on this error at: https://sqlalche.me/e/20/f405)


- This is successful because there is product 2 but it is not active

In [73]:
%sql INSERT INTO products (product_no, active) VALUES (2, true);

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.UndefinedColumn) HATA:  "active" kolonu "products" tablosunda mevcut değil
LINE 1: INSERT INTO products (product_no, active) VALUES (2, true);
                                          ^

[SQL: INSERT INTO products (product_no, active) VALUES (2, true);]
(Background on this error at: https://sqlalche.me/e/20/f405)


##### Using NULL values without DISTINCT restriction

In [74]:
%%sql
DROP TABLE IF EXISTS users;
CREATE TABLE users (
    email TEXT UNIQUE
);

 * postgresql://postgres@localhost/mydb
Done.
Done.


[]

- Inserting multiple NULL values does not throw error

In [75]:
%%sql
INSERT INTO users VALUES (NULL);
INSERT INTO users VALUES (NULL);

 * postgresql://postgres@localhost/mydb
1 rows affected.
1 rows affected.


[]

In [76]:
%sql SELECT * FROM users;

 * postgresql://postgres@localhost/mydb
2 rows affected.


email
None
None


##### Using NULL values with DISTINCT restriction

In [77]:
%%sql
DROP TABLE IF EXISTS users;
CREATE TABLE users (
    email TEXT UNIQUE NULLS NOT DISTINCT
);

 * postgresql://postgres@localhost/mydb
Done.
Done.


[]

- Violating UNIQUE constraint because NULLs are not distinct

In [78]:
%%sql
INSERT INTO users VALUES (NULL);
INSERT INTO users VALUES (NULL);

 * postgresql://postgres@localhost/mydb
1 rows affected.
(psycopg2.errors.UniqueViolation) HATA:  tekrar eden kayıt, "users_email_key" tekil kısıtlamasını ihlal etmektedir
DETAIL:  "(email)=(null)" anahtarı zaten mevcut

[SQL: INSERT INTO users VALUES (NULL);]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


#### 5.5.4. Primary Keys

- <i>PRIMARY KEY</i> can be defined for single column or column pairs.
    - <i>product_no integer PRIMARY KEY,</i>
    - <i>PRIMARY KEY(a,c)</i>
- Tables can have at most one primary key.
- Behaviour of <i>PRIMARY KEY</i> and <i>UNIQUE NOT NULL</i> is equivalent.
- Primary key creates B-tree index for interested column (or pair of columns).

##### Single column PRIMARY KEY

In [79]:
%%sql
DROP TABLE IF EXISTS products;
CREATE TABLE products (
    product_no integer PRIMARY KEY,
    name text,
    price numeric
);

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.DependentObjectsStillExist) HATA:  diğer nesnelerin ona bağlı olması nedeniyle products tablosu kaldırılamıyor
DETAIL:  orders tablosu üzerinde orders_product_no_fkey kısıtlaması, products tablosu nesnesine bağlıdır
HINT:  Bağlı nesneleri de kaldırmak için DROP ... CASCADE kullanın.

[SQL: DROP TABLE IF EXISTS products;]
(Background on this error at: https://sqlalche.me/e/20/2j85)


- See index for PK

In [80]:
%sql SELECT indexname, indexdef FROM pg_indexes WHERE tablename = 'products';

 * postgresql://postgres@localhost/mydb
1 rows affected.


indexname,indexdef
products_pkey,CREATE UNIQUE INDEX products_pkey ON public.products USING btree (product_no)


- Violation of PK constraint

In [81]:
%%sql
INSERT INTO products (product_no) VALUES (1);
INSERT INTO products (product_no) VALUES (1);

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.CheckViolation) HATA:  "products" tablosuna girilen yeni satır "lock_table" check kısıtlamasını ihlal ediyor
DETAIL:  Hata veren satır (1, null, null) içeriyor.

[SQL: INSERT INTO products (product_no) VALUES (1);]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


##### Pair of columns PRIMARY KEY

In [82]:
%%sql
DROP TABLE IF EXISTS example;
CREATE TABLE example (
    a integer,
    b integer,
    c integer,
    PRIMARY KEY (a,b)
);

 * postgresql://postgres@localhost/mydb
Done.
Done.


[]

- Inserting values

In [83]:
%%sql
INSERT INTO example (a,b) VALUES (1,1);
INSERT INTO example (a,b) VALUES (1,2);
INSERT INTO example (a,b) VALUES (2,1);

 * postgresql://postgres@localhost/mydb
1 rows affected.
1 rows affected.
1 rows affected.


[]

- Violation of PK

In [84]:
%sql INSERT INTO example (a,b) VALUES (1,1);

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.UniqueViolation) HATA:  tekrar eden kayıt, "example_pkey" tekil kısıtlamasını ihlal etmektedir
DETAIL:  "(a, b)=(1, 1)" anahtarı zaten mevcut

[SQL: INSERT INTO example (a,b) VALUES (1,1);]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


##### Adding PK with ALTER TABLE

In [85]:
%%sql
DROP TABLE IF EXISTS products;
CREATE TABLE products (
    product_no integer,
    name text,
    price numeric
);

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.DependentObjectsStillExist) HATA:  diğer nesnelerin ona bağlı olması nedeniyle products tablosu kaldırılamıyor
DETAIL:  orders tablosu üzerinde orders_product_no_fkey kısıtlaması, products tablosu nesnesine bağlıdır
HINT:  Bağlı nesneleri de kaldırmak için DROP ... CASCADE kullanın.

[SQL: DROP TABLE IF EXISTS products;]
(Background on this error at: https://sqlalche.me/e/20/2j85)


In [86]:
%sql ALTER TABLE products ADD CONSTRAINT pk_products_product_no PRIMARY KEY (product_no);

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.InvalidTableDefinition) HATA:  "products" tablosunda birden çok birincil anahtara izin verilmez

[SQL: ALTER TABLE products ADD CONSTRAINT pk_products_product_no PRIMARY KEY (product_no);]
(Background on this error at: https://sqlalche.me/e/20/f405)


#### 5.5.5. Foreign Keys

- <i>FOREIGN KEY</i> constraint specifies <i>referential integrity</i> between related tables.
- FK can be used with specifying column name or only specifying table name.
    - <i>product_no integer REFERENCES products (product_no)</i>
    - <i>product_no integer REFERENCES products</i>
- FOREIGN KEY does not create index automatically for interested column. To optimize querying on foreign key columns you can create an index.
- FK can be specified for non-primary columns. But referenced column must have UNIQUE constraint.
- You can reference same table in FK.
- A table can have multiple foreign key constraints.
    - This case can be used for many-to-many relationships.
- When deleting a record which have references there are 5 actions:
    - NO ACTION (default action):
        - Default action if there is is not any action specified is NO ACTION.
        - But you can define this action explicitly.
        - This action prevents deleting any record which has references on other tables.
    - RESTRICT (default action):
        - This action is like NO ACTION which prevents deleting any record which has references on other tables.
        - The difference between NO ACTION and RESTRICT is in transaction:
            - NO ACTION can be defined as deferrable.
                - <i>product_no integer REFERENCES products(product_no) ON DELETE NO ACTION DEFERRABLE INITIALLY DEFERRED,</i>
                - Waits until end of transaction to apply FK constraint.
            - RESTRICT can not be defined as deferrable.
                - PostgreSQL allows you to define RESTRICT with DEFERRABLE, but it is not effective. It still behaves as restricted and take action immediately.
    - CASCADE:
        - This action deletes record with reference records.
    - SET NULL:
        - This action sets value of referenced column as NULL.
    - SET DEFAULT:
        - This action sets value of referenced column as DEFAULT value.
        - Using SET DEFAULT is risky.
        - When setting default value of fk column, default value must reference any product otherwise error will be raised.

##### Usage of FOREIGN KEY constraint

- Defining FK constraint

In [87]:
%%sql
DROP TABLE IF EXISTS products, orders;
CREATE TABLE products (
    product_no integer PRIMARY KEY,
    name text,
    price numeric
);
CREATE TABLE orders (
    order_id integer PRIMARY KEY,
    product_no integer REFERENCES products (product_no),
    quantity integer
)

 * postgresql://postgres@localhost/mydb
Done.
Done.
Done.


[]

- Inserting product and related order 

In [88]:
%%sql
INSERT INTO products VALUES (1, 'product-1', 10);
INSERT INTO orders VALUES (1, 1, 10)

 * postgresql://postgres@localhost/mydb
1 rows affected.
1 rows affected.


[]

- Inserting order which is not related with any product

In [89]:
%sql INSERT INTO orders VALUES (2, NULL, 10)

 * postgresql://postgres@localhost/mydb
1 rows affected.


[]

- Inserting order with a product which not exists will throw error

In [90]:
%sql INSERT INTO orders VALUES (3, 2, 10)

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.ForeignKeyViolation) HATA:  "orders" tablosu üzerindeki ekleme veya güncelleme işlemi "orders_product_no_fkey" foreign key kısıtlamasını ihlal ediyor
DETAIL:  "products" tablosunda (product_no)=(2) anahtarı mevcut değildir.

[SQL: INSERT INTO orders VALUES (3, 2, 10)]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


##### Usage of FOREIGN KEY constraint as table constraint

In [91]:
%%sql
DROP TABLE IF EXISTS products, orders;
CREATE TABLE products (
    product_no integer PRIMARY KEY,
    name text,
    price numeric
);
CREATE TABLE orders (
    order_id integer PRIMARY KEY,
    product_no integer ,
    quantity integer,
    FOREIGN KEY (product_no) REFERENCES products(product_no)
)

 * postgresql://postgres@localhost/mydb
Done.
Done.
Done.


[]

##### Usage of FOREIGN KEY constraint for non-primary columns

- For non-unique referenced columns, error will be thrown

In [92]:
%%sql
DROP TABLE IF EXISTS products, orders;
CREATE TABLE products (
    product_no integer,
    name text,
    price numeric
);
CREATE TABLE orders (
    order_id integer PRIMARY KEY,
    product_no integer ,
    quantity integer,
    FOREIGN KEY (product_no) REFERENCES products(product_no)
)

 * postgresql://postgres@localhost/mydb
Done.
Done.
(psycopg2.errors.InvalidForeignKey) HATA:  "products" referans edilen tablosunda belirtilen anahtarlara uyan bir unique constraint yok

[SQL: CREATE TABLE orders (
    order_id integer PRIMARY KEY,
    product_no integer ,
    quantity integer,
    FOREIGN KEY (product_no) REFERENCES products(product_no)
)]
(Background on this error at: https://sqlalche.me/e/20/f405)


- For unique referenced columns, adding fk constraint will be successful

In [93]:
%%sql
DROP TABLE IF EXISTS products, orders;
CREATE TABLE products (
    product_no integer UNIQUE,
    name text,
    price numeric
);
CREATE TABLE orders (
    order_id integer PRIMARY KEY,
    product_no integer ,
    quantity integer,
    FOREIGN KEY (product_no) REFERENCES products(product_no)
)

 * postgresql://postgres@localhost/mydb
Done.
Done.
Done.


[]

- In contrast to a primary key column reference, when using a non-primary column for reference, you should specify the column name.
- In that case, an error will be thrown.

In [94]:
%%sql
DROP TABLE IF EXISTS products, orders;
CREATE TABLE products (
    product_no integer UNIQUE,
    name text,
    price numeric
);
CREATE TABLE orders (
    order_id integer PRIMARY KEY,
    product_no integer ,
    quantity integer,
    FOREIGN KEY (product_no) REFERENCES products
)

 * postgresql://postgres@localhost/mydb
Done.
Done.
(psycopg2.errors.UndefinedObject) HATA:  referans edilen "products" tablosunda primary key mevcut değil

[SQL: CREATE TABLE orders (
    order_id integer PRIMARY KEY,
    product_no integer ,
    quantity integer,
    FOREIGN KEY (product_no) REFERENCES products
)]
(Background on this error at: https://sqlalche.me/e/20/f405)


##### Usage of FOREIGN KEY for pair of columns

In [95]:
%%sql
DROP TABLE IF EXISTS t1, t2;
CREATE TABLE t1 (
    a integer,
    b integer,
    PRIMARY KEY (a,b)
);
CREATE TABLE t2 (
    q integer PRIMARY KEY,
    w integer,
    e integer,
    FOREIGN KEY (w, e) REFERENCES t1 (a, b)
);

 * postgresql://postgres@localhost/mydb
Done.
Done.
Done.


[]

- Inserting records for referenced tables

In [96]:
%%sql
INSERT INTO t1 VALUES (1,1);
INSERT INTO t1 VALUES (1,2);
INSERT INTO t1 VALUES (2,1);
INSERT INTO t2 VALUES (1,1,1);

 * postgresql://postgres@localhost/mydb
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

- Inserting record that there is no related record for referenced table will throw error

In [97]:
%sql INSERT INTO t2 VALUES (2,1,3);

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.ForeignKeyViolation) HATA:  "t2" tablosu üzerindeki ekleme veya güncelleme işlemi "t2_w_e_fkey" foreign key kısıtlamasını ihlal ediyor
DETAIL:  "t1" tablosunda (w, e)=(1, 3) anahtarı mevcut değildir.

[SQL: INSERT INTO t2 VALUES (2,1,3);]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


##### Self referencing in FOREIGN KEY

- A table column with a foreign key can reference the same table.

In [98]:
%%sql
DROP TABLE IF EXISTS tree;
CREATE TABLE tree (
    node_id integer PRIMARY KEY,
    parent_id integer REFERENCES tree,
    name text
);

 * postgresql://postgres@localhost/mydb
Done.
Done.


[]

- In this example parent of root node will be null

In [99]:
%%sql
INSERT INTO tree VALUES (1, NULL, 'root');
INSERT INTO tree VALUES (2, 1, 'child-1 of 1');
INSERT INTO tree VALUES (3, 1, 'child-2 of 2');

 * postgresql://postgres@localhost/mydb
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [100]:
%sql SELECT * FROM tree;

 * postgresql://postgres@localhost/mydb
3 rows affected.


node_id,parent_id,name
1,None,root
2,1,child-1 of 1
3,1,child-2 of 2


##### Multiple FOREIGN KEY constraints

- When to establish many-to-many relationship for order and products, a middle table with referencing this tables can be used.

In [101]:
%%sql
DROP TABLE IF EXISTS products, orders, order_items;
CREATE TABLE products (
    product_no integer PRIMARY KEY,
    name text,
    price numeric
);
CREATE TABLE orders (
    order_id integer PRIMARY KEY,
    shipping_address text
);
CREATE TABLE order_items (
    product_no integer REFERENCES products,
    order_id integer REFERENCES orders,
    quantity integer,
    PRIMARY KEY (product_no, order_id)
);

 * postgresql://postgres@localhost/mydb
Done.
Done.
Done.
Done.


[]

##### Actions when deleting referenced records

###### Preventing delete action with NO ACTION (Default action)

- When not specifying delete action, it is automatically assigned as NO ACTION
- But you can specify explicitly NO ACTION

In [102]:
%%sql
DROP TABLE IF EXISTS products, orders CASCADE;
CREATE TABLE products (
    product_no integer PRIMARY KEY,
    name text,
    price numeric
);
CREATE TABLE orders (
    order_id integer PRIMARY KEY,
    product_no integer REFERENCES products(product_no) ON DELETE NO ACTION,
    quantity integer
);

 * postgresql://postgres@localhost/mydb
Done.
Done.
Done.


[]

- Inserting referenced records

In [103]:
%%sql
INSERT INTO products VALUES (1, 'product-1', 10);
INSERT INTO orders VALUES (1, 1, 10);

 * postgresql://postgres@localhost/mydb
1 rows affected.
1 rows affected.


[]

- When trying to delete a product which referenced in orders will raise error

In [104]:
%sql DELETE FROM products WHERE product_no = 1;

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.ForeignKeyViolation) HATA:  "products" tablosu üzerinde yapılan update veya delete işlemi "orders" tablosunun "orders_product_no_fkey" bütünlük kısıtlamasını ihlal ediyor
DETAIL:  (product_no)=(1) anahtarı "orders" tablosundan hala referans edilmektedir.

[SQL: DELETE FROM products WHERE product_no = 1;]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


- Deleting related orders before deleting product will be successful

In [105]:
%%sql 
DELETE FROM orders WHERE order_id = 1;
DELETE FROM products WHERE product_no = 1;

 * postgresql://postgres@localhost/mydb
1 rows affected.
1 rows affected.


[]

###### Preventing delete action with RESTRICT

In [106]:
%%sql
DROP TABLE IF EXISTS products, orders CASCADE;
CREATE TABLE products (
    product_no integer PRIMARY KEY,
    name text,
    price numeric
);
CREATE TABLE orders (
    order_id integer PRIMARY KEY,
    product_no integer REFERENCES products(product_no) ON DELETE RESTRICT,
    quantity integer
);

 * postgresql://postgres@localhost/mydb
Done.
Done.
Done.


[]

- Inserting referenced records

In [107]:
%%sql
INSERT INTO products VALUES (1, 'product-1', 10);
INSERT INTO orders VALUES (1, 1, 10);

 * postgresql://postgres@localhost/mydb
1 rows affected.
1 rows affected.


[]

- When trying to delete a product which referenced in orders will raise error

In [108]:
%sql DELETE FROM products WHERE product_no = 1;

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.ForeignKeyViolation) HATA:  "products" tablosu üzerinde yapılan update veya delete işlemi "orders" tablosunun "orders_product_no_fkey" bütünlük kısıtlamasını ihlal ediyor
DETAIL:  (product_no)=(1) anahtarı "orders" tablosundan hala referans edilmektedir.

[SQL: DELETE FROM products WHERE product_no = 1;]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


- Deleting related orders before deleting product will be successful

In [109]:
%%sql 
DELETE FROM orders WHERE order_id = 1;
DELETE FROM products WHERE product_no = 1;

 * postgresql://postgres@localhost/mydb
1 rows affected.
1 rows affected.


[]

###### Deleting related records automatically with CASCADE

In [110]:
%%sql
DROP TABLE IF EXISTS products, orders CASCADE;
CREATE TABLE products (
    product_no integer PRIMARY KEY,
    name text,
    price numeric
);
CREATE TABLE orders (
    order_id integer PRIMARY KEY,
    product_no integer REFERENCES products(product_no) ON DELETE CASCADE,
    quantity integer
);

 * postgresql://postgres@localhost/mydb
Done.
Done.
Done.


[]

- Inserting related records

In [111]:
%%sql
INSERT INTO products VALUES (1, 'product-1', 10);
INSERT INTO orders VALUES (1, 1, 10);

 * postgresql://postgres@localhost/mydb
1 rows affected.
1 rows affected.


[]

In [112]:
%sql SELECT * FROM products;

 * postgresql://postgres@localhost/mydb
1 rows affected.


product_no,name,price
1,product-1,10


In [113]:
%sql SELECT * FROM orders;

 * postgresql://postgres@localhost/mydb
1 rows affected.


order_id,product_no,quantity
1,1,10


- Deleting product which has related orders will cause deleting orders

In [114]:
%sql DELETE FROM products WHERE product_no = 1;

 * postgresql://postgres@localhost/mydb
1 rows affected.


[]

- See that both product and order is deleted

In [115]:
%sql SELECT * from products;

 * postgresql://postgres@localhost/mydb
0 rows affected.


product_no,name,price


In [116]:
%sql SELECT * FROM orders;

 * postgresql://postgres@localhost/mydb
0 rows affected.


order_id,product_no,quantity


###### Deleting related records automatically with set NULL

In [117]:
%%sql
DROP TABLE IF EXISTS products, orders CASCADE;
CREATE TABLE products (
    product_no integer PRIMARY KEY,
    name text,
    price numeric
);
CREATE TABLE orders (
    order_id integer PRIMARY KEY,
    product_no integer REFERENCES products(product_no) ON DELETE SET NULL,
    quantity integer
);

 * postgresql://postgres@localhost/mydb
Done.
Done.
Done.


[]

- Inserting related records

In [118]:
%%sql
INSERT INTO products VALUES (1, 'product-1', 10);
INSERT INTO orders VALUES (1, 1, 10);

 * postgresql://postgres@localhost/mydb
1 rows affected.
1 rows affected.


[]

- Deleting product will cause setting product_no of related order to NULL

In [119]:
%sql DELETE FROM products WHERE product_no = 1;

 * postgresql://postgres@localhost/mydb
1 rows affected.


[]

- See that product is deleted but order still exists

In [120]:
%sql SELECT * FROM products;

 * postgresql://postgres@localhost/mydb
0 rows affected.


product_no,name,price


In [121]:
%sql SELECT * FROM orders;

 * postgresql://postgres@localhost/mydb
1 rows affected.


order_id,product_no,quantity
1,None,10


###### Deleting related records automatically with SET DEFAULT

- Using SET DEFAULT is risky.
- When setting default value of fk column, default value must reference any product otherwise error will be raised.

In [122]:
%%sql
DROP TABLE IF EXISTS products, orders CASCADE;
CREATE TABLE products (
    product_no integer PRIMARY KEY,
    name text,
    price numeric
);
CREATE TABLE orders (
    order_id integer PRIMARY KEY,
    product_no integer DEFAULT 1 REFERENCES products(product_no) ON DELETE SET DEFAULT,
    quantity integer
);

 * postgresql://postgres@localhost/mydb
Done.
Done.
Done.


[]

- Inserting related records

In [123]:
%%sql
INSERT INTO products VALUES (1, 'product-1', 10);
INSERT INTO products VALUES (2, 'product-2', 10);
INSERT INTO orders VALUES (1, 1, 10);
INSERT INTO orders VALUES (2, 2, 10);

 * postgresql://postgres@localhost/mydb
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [124]:
%sql SELECT * FROM products;

 * postgresql://postgres@localhost/mydb
2 rows affected.


product_no,name,price
1,product-1,10
2,product-2,10


In [125]:
%sql SELECT * FROM orders;

 * postgresql://postgres@localhost/mydb
2 rows affected.


order_id,product_no,quantity
1,1,10
2,2,10


- Deleting product with id 2 will cause the change of order with product_no 2.
- New product_no value of order-2 will be 1. Since there is a product with product_no 1 there will be no error.

In [126]:
%sql DELETE FROM products WHERE product_no = 2;

 * postgresql://postgres@localhost/mydb
1 rows affected.


[]

In [127]:
%sql SELECT * FROM products;

 * postgresql://postgres@localhost/mydb
1 rows affected.


product_no,name,price
1,product-1,10


In [128]:
%sql SELECT * FROM orders;

 * postgresql://postgres@localhost/mydb
2 rows affected.


order_id,product_no,quantity
1,1,10
2,1,10


- Now deleting product with id 1 will cause the change of orders with product_no 1.
- But since there is no record in products with product_no 1 an error will be raised.

In [129]:
%sql DELETE FROM products WHERE product_no = 1;

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.ForeignKeyViolation) HATA:  "products" tablosu üzerinde yapılan update veya delete işlemi "orders" tablosunun "orders_product_no_fkey" bütünlük kısıtlamasını ihlal ediyor
DETAIL:  (product_no)=(1) anahtarı "orders" tablosundan hala referans edilmektedir.

[SQL: DELETE FROM products WHERE product_no = 1;]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


###### Difference between NO ACTION and RESTRICTED

- NO ACTION can be described as DEFERRABLE but restricted can not.

- To see the effects in this section auto-commit will be disabled.
- Do not forget to re-open for other sections.

In [130]:
%config SqlMagic.autocommit=False

In [131]:
%%sql
BEGIN;
DROP TABLE IF EXISTS products, orders CASCADE;
CREATE TABLE products (
    product_no integer PRIMARY KEY,
    name text,
    price numeric
);
CREATE TABLE orders (
    order_id integer PRIMARY KEY,
    product_no integer REFERENCES products(product_no) ON DELETE NO ACTION DEFERRABLE INITIALLY DEFERRED,
    quantity integer
);
COMMIT;

 * postgresql://postgres@localhost/mydb
Done.
Done.
Done.
Done.
Done.


[]

- Inserting related records

In [132]:
%%sql
BEGIN;
INSERT INTO products VALUES (1, 'product-1', 10);
INSERT INTO orders VALUES (1, 1, 10);
COMMIT;

 * postgresql://postgres@localhost/mydb
Done.
1 rows affected.
1 rows affected.
Done.


[]

- In transaction first product then order can be deleted because FK check is deferred until transaction ends
- See that both product and order deleted

In [133]:
%%sql
BEGIN;
DELETE FROM products WHERE product_no = 1;
DELETE FROM orders WHERE product_no = 1;
COMMIT;

 * postgresql://postgres@localhost/mydb
Done.
1 rows affected.
1 rows affected.
Done.


[]

In [134]:
%sql SELECT * FROM products;

 * postgresql://postgres@localhost/mydb
0 rows affected.


product_no,name,price


In [135]:
%sql SELECT * FROM orders;

 * postgresql://postgres@localhost/mydb
0 rows affected.


order_id,product_no,quantity


- Even when defining RESTRICT as DEFERRED, it has no effect on transaction.
- Still deleting product before delete order violates error even in transaction.
- So it behaves as RESTRICT even if you define it as DEFERRED.

In [136]:
%%sql
BEGIN;
DROP TABLE IF EXISTS products, orders CASCADE;
CREATE TABLE products (
    product_no integer PRIMARY KEY,
    name text,
    price numeric
);
CREATE TABLE orders (
    order_id integer PRIMARY KEY,
    product_no integer REFERENCES products(product_no) ON DELETE RESTRICT DEFERRABLE INITIALLY DEFERRED,
    quantity integer
);
COMMIT;

 * postgresql://postgres@localhost/mydb
Done.
Done.
Done.
Done.
Done.


[]

- Inserting related records

In [137]:
%%sql
BEGIN;
INSERT INTO products VALUES (1, 'product-1', 10);
INSERT INTO orders VALUES (1, 1, 10);
COMMIT;

 * postgresql://postgres@localhost/mydb
Done.
1 rows affected.
1 rows affected.
Done.


[]

- Delete product will raise error because FK constraint checks violation imeediately even if described as DEFERRED

In [138]:
%%sql
BEGIN;
DELETE FROM products WHERE product_no = 1;
DELETE FROM orders WHERE product_no = 1;
COMMIT;

 * postgresql://postgres@localhost/mydb
Done.
(psycopg2.errors.ForeignKeyViolation) HATA:  "products" tablosu üzerinde yapılan update veya delete işlemi "orders" tablosunun "orders_product_no_fkey" bütünlük kısıtlamasını ihlal ediyor
DETAIL:  (product_no)=(1) anahtarı "orders" tablosundan hala referans edilmektedir.

[SQL: DELETE FROM products WHERE product_no = 1;]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


In [139]:
%config SqlMagic.autocommit=True

#### 5.5.6. Exclusion Constraints

- Exclusion constraint works like UNIQUE operator but it allows compare values to constraint other than equality operator.
- Normally, a UNIQUE constraint prevents duplicate values in a column.
- An Exclusion Constraint, on the other hand, enforces a "cannot exist at the same time" restriction using a specified operator.
- Usage is  <i>EXCLUDE USING gist (point(lat, lon) WITH ~=)</i>
    - When using exclude column will be indexed.
    - In here gist specifies Generalized Search Tree algorithm will be used in indexing.
    - Other alternatives: btree, spgist, brin.
    - But gist is more common.

In [140]:
%%sql
DROP TABLE IF EXISTS locations;
CREATE TABLE locations (
    id SERIAL PRIMARY KEY,
    lat DOUBLE PRECISION NOT NULL,
    lon DOUBLE PRECISION NOT NULL,
    EXCLUDE USING gist (point(lat, lon) WITH ~=)
);

 * postgresql://postgres@localhost/mydb
Done.
Done.


[]

- Inserting same point will raise exception

In [141]:
%%sql
INSERT INTO locations VALUES (1, 10.0, 15.0);
INSERT INTO locations VALUES (2, 10.0, 15.0);

 * postgresql://postgres@localhost/mydb
1 rows affected.
(psycopg2.errors.ExclusionViolation) HATA:  uyuşmayan kayıt, "locations_point_excl" exclusion kısıtlamasını ihlal etmektedir
DETAIL:  (point(lat, lon))=((10,15)) anahtarı, mevcut (point(lat, lon))=((10,15)) anahtarıyla uyuşmuyor.

[SQL: INSERT INTO locations VALUES (2, 10.0, 15.0);]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


- Another example using date range

In [142]:
%%sql
CREATE EXTENSION IF NOT EXISTS btree_gist; 
DROP TABLE IF EXISTS reservations;
CREATE TABLE reservations (
    id SERIAL PRIMARY KEY,
    room_id INTEGER NOT NULL,
    start_date DATE NOT NULL,
    end_date DATE NOT NULL,
    EXCLUDE USING gist (
        room_id WITH =,  -- Aynı oda numarasının tekrarını engeller
        daterange(start_date, end_date, '[]') WITH &&  -- Tarihler çakışamaz
    )
);

 * postgresql://postgres@localhost/mydb
Done.
Done.
Done.


[]

- Inserting reservations

In [143]:
%%sql
INSERT INTO reservations (room_id, start_date, end_date) VALUES (1, '2025-03-01', '2025-03-04');
INSERT INTO reservations (room_id, start_date, end_date) VALUES (1, '2025-03-05', '2025-03-07');

 * postgresql://postgres@localhost/mydb
1 rows affected.
1 rows affected.


[]

- Inserting a record which dates collide with another record will raise error

In [144]:
%sql INSERT INTO reservations (room_id, start_date, end_date) VALUES (1, '2025-03-06', '2025-03-06');

 * postgresql://postgres@localhost/mydb
(psycopg2.errors.ExclusionViolation) HATA:  uyuşmayan kayıt, "reservations_room_id_daterange_excl" exclusion kısıtlamasını ihlal etmektedir
DETAIL:  (room_id, daterange(start_date, end_date, '[]'::text))=(1, [2025-03-06,2025-03-07)) anahtarı, mevcut (room_id, daterange(start_date, end_date, '[]'::text))=(1, [2025-03-05,2025-03-08)) anahtarıyla uyuşmuyor.

[SQL: INSERT INTO reservations (room_id, start_date, end_date) VALUES (1, '2025-03-06' , '2025-03-06' );]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


### 5.7. Modifying Tables